# Challenge data - Cours de STAT - Centrale Nantes
Le but du projet est d'être capable de prédire le sexe d'une personne à partir de mesure de ses EEG durant une période d'endormissement.

Le notebook ci-présent ne comporte que le strict minimum de lignes nécessaires à entraîner notre meilleur modèle et à classifier les données de test. **Toutes les fonctions qui nous ont servie à effectuer des tests tout au long du projet se trouve sur [ce repo github](https://github.com/jean-tissot/stat_challenge_data_ens "Notre repo github (https://github.com/jean-tissot/stat_challenge_data_ens)")**

## Importation des modules nécessaires

In [1]:
import h5py,pandas as pd, csv
import matplotlib.pyplot as plt, numpy as np
import os.path,sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from tensorflow import keras
from tensorflow.keras import layers

## Aquisition des données:  
*X_train* est le dataset d'entrainement, de taille (946, 40, 7, 500) (classe correspondantes dans *y_train*, de taille (946,))  
*X_test* est le dataset à classifier, de taille (946, 40, 7, 500)

In [ ]:
X_file = h5py.File("data/X_train_new.h5",'r') # X est ici au format "HDF5 dataset"
X_train = np.array(X_file['features']) # X est ici au format "ndarray"

X_final_file = h5py.File("data/X_test_new.h5",'r') # X est ici au format "HDF5 dataset"
X_test = np.array(X_final_file['features']) # X est ici au "ndarray"

y = pd.read_csv("data/y_train_AvCsavx.csv") # y est ici au format "pandas DataFrame"
y_train = np.array(y['label']) # y est ici au format "ndarray"

## Préprocessing des données
Nous effectuons une standardisation des datasets (cela accélère la descente de gradient et limite les oscillations de l'accuracy évoluant au fur et à mesure de l'entraînement)

In [3]:
transform = StandardScaler().fit_transform #fonction de standardisation
for j in range(40):
    for i in range(X_train.shape[0]):
        X_train[i,j,:,:] = np.transpose(transform(np.transpose(X_train[i,j,:,:])))
    for i in range(X_test.shape[0]):
        X_test[i,j,:,:] = np.transpose(transform(np.transpose(X_test[i,j,:,:])))

Le dataset d'entraînement est composé de 946 personnes, et pour chacune on dispose de 40 fenêtre de 2 secondes d'enregistrement (sur 7 canaux) d'EEG. Les 40 fenêtres étant **indépendantes** il est judicieux de les séparer pour l'entrainement et de les considérer comme 40 personnes différentes.  
Notre dataset équivaut donc à un dataset de 40*946 personnes pour lesquelles on dispose d'une fenêtre de 2 seconde d'enregistrement d'EEG sur 7 canaux.

In [5]:
X_train=np.concatenate(X_train, axis=0)  #sépération des 40 fenêtres indépendantes (comme si chaque fenêtre correspondait à une personne)
y_train=np.repeat(y_train, 40)  #Multiplication par 40 de chaque personne (car séparation des fenêtres)
X_train, y_train = shuffle(X_train, y_train) #mélange des données pour ne pas avoir 40 fois le même sexe d'affilé

## Rééquilibrage du dataset de training
Le dataset d'entraînement comporte environ 77% d'hommes, ce qui déséquilibre l'entrainement et génère un modèle qui a tendance à prédire uniquement des hommes. Nous devons donc rééquilibrer ce dataset.

- La simple suppression de données concernant des hommes réduit drastiquement la taille du dataset, ce que nous ne pouvons nous permettre si nous voulons avoir un modèle assez entraîné (et ne pas faire d'overfitting)
- La génération de données synthétique concernant les femmes induit un facteur trop grand de données synthétisées par rapport aux données réelles (environ 2 fois plus de données synthétisée que de données réelles concernant les femmes)
- Nous combinons donc ces deux méthodes:
Suppression d'une partie des hommes et synthèse de données concernant les femmes par duplication (la méthode SMOTE n'ayant pas donné de bons résultats)
### Récupération des indices des hommes et des femmes, ainsi que leurs nombres respectifs

In [1]:
mask_h = np.where(y_train==0)[0] #indices de tous les hommes
mask_f = np.where(y_train==1)[0] #indices de toutes les femmes
nb_h = len(mask_h) #nombre d'hommes
nb_f = len(mask_f) #nombre de femmes

### Génération d'une liste d'indices d'hommes à supprimer et de femmes à ajouter

In [ ]:
nb = (nb_h - nb_f)//2
mask_h = mask_h[:nb] #liste de taille (nb_h - nb_f)/2 d'indices d'hommes ) à supprimer
mask_f = np.resize(mask_f, nb) #liste de taille (nb_h - nb_f)/2 d'indices de femmes à rajouter (potentiellement répétés)

### Supression d'hommes et ajout de femmes

In [ ]:
add_X = X_train[mask_f,...] #liste des femmes à rajouter
add_y = y_train[mask_f]
X_train = np.delete(X_train, mask_h, 0) #suppression de la liste des hommes
y_train = np.delete(y_train, mask_h, 0)
X_train=np.concatenate((X_train, add_X), axis=0) #ajout de la liste de femmes
y_train=np.concatenate((y_train, add_y), axis=0)

## Création du CNN
Ce CNN correspond au modèle que l'on a retenu pour le projet. Les autres modèles testés sont disponibles sur [le repo github](https://github.com/jean-tissot/stat_challenge_data_ens "Notre repo github (https://github.com/jean-tissot/stat_challenge_data_ens)") cité au début de ce notebook.

In [ ]:
model = keras.Sequential(
[
    layers.Conv2D(filters=25, kernel_size=(1,10), strides=1, padding='same', input_shape=(7,500,1)),
    layers.Conv2D(filters=25, kernel_size=(7,1), strides=1, padding='same', use_bias=False),
    layers.BatchNormalization(momentum=0.1, epsilon=0.00001),
    layers.Activation('elu'),
    layers.MaxPool2D(pool_size=(1,3), strides=(1,3), padding='same'),
    
    layers.Dropout(0.5),
    layers.Conv2D(filters=50, kernel_size=(1,10), strides=1, padding='same', use_bias=False),
    layers.BatchNormalization(momentum=0.1, epsilon=0.00001),
    layers.Activation('elu'),
    layers.MaxPool2D(pool_size=(1,3), strides=(1,3), padding='same'),

    layers.Dropout(0.5),
    layers.Conv2D(filters=100, kernel_size=(1,10), strides=1, padding='same', use_bias=False),
    layers.BatchNormalization(momentum=0.1, epsilon=0.00001),
    layers.Activation('elu'),
    layers.MaxPool2D(pool_size=(1,3), strides=(1,3), padding='same'),

    layers.Dropout(0.5),
    layers.Conv2D(filters=200, kernel_size=(1,10), strides=1, padding='same', use_bias=False),
    layers.BatchNormalization(momentum=0.1, epsilon=0.00001),
    layers.Activation('elu'),
    layers.MaxPool2D(pool_size=(1,3), strides=(1,3), padding='same'),

    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
]
)

Le CNN utilise une fonction de perte de type *binary_crossentropy* et un optimizer de type *adamax*

In [ ]:
model.compile(
loss='binary_crossentropy',
optimizer=keras.optimizers.Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
metrics=[keras.metrics.BinaryAccuracy(name='accuracy'), keras.metrics.AUC(name='AUC')]
)
model.summary()

## Entrainement du modèle
- 60 passes
- Chaque est effectuée sur l'ensemble des données, découpé en groupes de 69 éléments

In [ ]:
epochs=60
batch_size=69
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)


## Application du modèle aux données à classifier
Il faut découper chaque donnée à classifier en 40 fenêtres distinctes de 2 secondes (puisque notre modèle a été conçu pour des fenêtres isolées).  
Pour effectuer la prédiction du sexe d'une personne, nous effectuons une moyenne des prédictions sur chacune des 40 fenêtres de 2 secondes d'enregistrement de ses EEG.

In [ ]:
y_pred=[]
for i in range(40):
    X_test_i=X_test[:,i,:,:]
    X_test_i=X_test_i.reshape(X_test_i.shape[0], X_test_i.shape[1], X_test_i.shape[2], 1)
    y_pred.append(model.predict(X_test_i))
y_pred=np.mean(y_pred, axis=0)

## Export de la classification au format CSV

In [ ]:
with open('classification.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "label"])
    for i in range(len(y_pred)):
        writer.writerow([i, int(y_pred[i]>0.5)])